## <b>4.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>4.3.1 Integració i neteja de dades</b>

#### <b>4.3.1.1 Càrrega i verificació inicial de les dades</b>

In [4]:
# ============================================================
# Càrrega i verificació inicial de les dades
# ============================================================
# Objectiu:
#   - Carregar el dataset principal (CSV) i el diccionari (XLSX)
#   - Verificar dimensions, estructura, tipus, duplicats i nuls
#   - Comprovar coherència entre variables del dataset i del diccionari
#   - Generar un resum inicial auditable
#
# Nota:
#   Aquest codi només fa CÀRREGA + VERIFICACIÓ INICIAL.
#   No fa imputacions ni transformacions.
# ============================================================
import pandas as pd  # Llibreria principal per manipulació de dades
import numpy as np   # Llibreria per operacions numèriques (no usada directament però importada com a utilitat estàndard)

# ------------------------------------------------------------
# 0) Rutes dels fitxers (ajusta-les si cal)
# ------------------------------------------------------------
DATA_PATH = "Motor vehicle insurance data.csv"
DICT_PATH = "Descriptive of the variables.xlsx"

# ------------------------------------------------------------
# 1) Càrrega robusta del CSV
# ------------------------------------------------------------
# Intentem llegir amb separador automàtic.
# Si el fitxer està separat per ';', detectarà correctament.
# Si està separat per ',', també.
df = pd.read_csv(DATA_PATH, sep=None, engine="python")
print("Dataset carregat correctament")

# ------------------------------------------------------------
# 2) Càrrega del diccionari de variables (Excel)
# ------------------------------------------------------------
# Carreguem totes les fulles per poder inspeccionar.
dict_xls = pd.ExcelFile(DICT_PATH)
print("\nFulls disponibles al diccionari:", dict_xls.sheet_names)
dict_df = pd.read_excel(DICT_PATH, sheet_name=dict_xls.sheet_names[0])
print("\nDiccionari carregat correctament")

# ------------------------------------------------------------
# 3) Verificació inicial del dataset
# ------------------------------------------------------------
# 3.1 Dimensions
print("\nDimensions del dataset (files, columnes):", df.shape)
# 3.2 Mostra de dades
print("\nPrimeres 5 files del dataset:")
display(df.head())
# 3.3 Noms de columnes
print("\nColumnes del dataset:")
print(df.columns.tolist())
# 3.4 Tipus de dades
print("\nTipus de dades per columna:")
print(df.dtypes)
# Distribució general de tipus
print("\nDistribució de tipus:")
print(df.dtypes.value_counts())

# ------------------------------------------------------------
# 4) Coherència amb el diccionari de variables
# ------------------------------------------------------------
print("\nDimensions del diccionari:", dict_df.shape)
print("\nPrimeres files del diccionari:")
display(dict_df.head())

# Agafem el camp de noms de variables del diccionari.
dict_vars = set(dict_df["Variables"].astype(str).str.strip())
data_vars = set(df.columns.astype(str).str.strip())
vars_in_dict_not_data = dict_vars - data_vars
vars_in_data_not_dict = data_vars - dict_vars
print("\nVariables al diccionari però no al dataset:", vars_in_dict_not_data)
print("Variables al dataset però no al diccionari:", vars_in_data_not_dict)

# ------------------------------------------------------------
# 5) Duplicats i estructura panel
# ------------------------------------------------------------
# 5.1 Duplicats exactes
n_dup_rows = df.duplicated().sum()
print("\nNombre de files duplicades exactes:", n_dup_rows)
# 5.2 Unicitat d'ID (esperem repetició per anualitats)
n_unique_ids = df["ID"].nunique()
pct_repeated = 1 - (n_unique_ids / len(df))
print("Nombre d'IDs únics:", n_unique_ids)
print("Percentatge d'IDs repetits (panel):", round(pct_repeated, 4))

# ------------------------------------------------------------
# 6) Taula de valors nuls
# ------------------------------------------------------------
missing_count = df.isna().sum()
missing_pct = (missing_count / len(df) * 100).round(2)
missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_pct": missing_pct
}).sort_values("missing_count", ascending=False)
print("\nTaula de valors nuls (top 10):")
display(missing_table.head(10))

# Variables amb >10% de nuls
high_missing_vars = missing_table[missing_table["missing_pct"] > 10].index.tolist()
print("Variables amb >10% nuls:", high_missing_vars)

# ------------------------------------------------------------
# 7) Rangs bàsics de variables numèriques
# ------------------------------------------------------------
num_cols = df.select_dtypes(include=[np.number]).columns
ranges = df[num_cols].agg(["min", "max"]).T
print("\nRangs mínim i màxim de variables numèriques:")
display(ranges)

# ------------------------------------------------------------
# 8) Resum executiu inicial
# ------------------------------------------------------------
summary = {
    "Registres": len(df),
    "Variables": df.shape[1],
    "Duplicats exactes": int(n_dup_rows),
    "Variables amb >10% nuls": high_missing_vars
}
print("\n--- RESUM INICIAL ---")
for k, v in summary.items():
    print(f"{k}: {v}")

# Guardem df i dict_df per als següents subapartats
# df  -> dataset principal
# dict_df -> diccionari de variables


Dataset carregat correctament

Fulls disponibles al diccionari: ['Motor vehicle insurance data', 'sample type claim']

Diccionari carregat correctament

Dimensions del dataset (files, columnes): (105555, 30)

Primeres 5 files del dataset:


,ID,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Distribution_channel,Seniority,Policies_in_force,Max_policies,...,Area,Second_driver,Year_matriculation,Power,Cylinder_capacity,Value_vehicle,N_doors,Type_fuel,Length,Weight
0,1,05/11/2015,05/11/2015,05/11/2016,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
1,1,05/11/2015,05/11/2016,05/11/2017,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
2,1,05/11/2015,05/11/2017,05/11/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
3,1,05/11/2015,05/11/2018,05/11/2019,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
4,2,26/09/2017,26/09/2017,26/09/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190



Columnes del dataset:
['ID', 'Date_start_contract', 'Date_last_renewal', 'Date_next_renewal', 'Date_birth', 'Date_driving_licence', 'Distribution_channel', 'Seniority', 'Policies_in_force', 'Max_policies', 'Max_products', 'Lapse', 'Date_lapse', 'Payment', 'Premium', 'Cost_claims_year', 'N_claims_year', 'N_claims_history', 'R_Claims_history', 'Type_risk', 'Area', 'Second_driver', 'Year_matriculation', 'Power', 'Cylinder_capacity', 'Value_vehicle', 'N_doors', 'Type_fuel', 'Length', 'Weight']

Tipus de dades per columna:
ID                        int64
Date_start_contract      object
Date_last_renewal        object
Date_next_renewal        object
Date_birth               object
Date_driving_licence     object
Distribution_channel      int64
Seniority                 int64
Policies_in_force         int64
Max_policies              int64
Max_products              int64
Lapse                     int64
Date_lapse               object
Payment                   int64
Premium                 flo

,Variables,Description
0,ID,Internal identification number assigned to eac...
1,Date_start _contract,Start date of the policyholder's contract (DD/...
2,Date_last_renewal,Date of last contract renewal (DD/MM/YYYY).
3,Date_next_renewal,Date of the next contract renewal (DD/MM/YYYY).
4,Distribution_channel,Classifies the channel through which the polic...



Variables al diccionari però no al dataset: {'Date_start _contract'}
Variables al dataset però no al diccionari: {'Date_start_contract'}

Nombre de files duplicades exactes: 0
Nombre d'IDs únics: 53502
Percentatge d'IDs repetits (panel): 0.4931

Taula de valors nuls (top 10):


,missing_count,missing_pct
Date_lapse,70408,66.70
Length,10329,9.79
Type_fuel,1764,1.67
ID,0,0.00
N_claims_year,0,0.00
N_doors,0,0.00
Value_vehicle,0,0.00
Cylinder_capacity,0,0.00
Power,0,0.00
Year_matriculation,0,0.00


Variables amb >10% nuls: ['Date_lapse']

Rangs mínim i màxim de variables numèriques:


,min,max
ID,1.000,53502.000
Distribution_channel,0.000,1.000
Seniority,1.000,40.000
Policies_in_force,1.000,17.000
Max_policies,1.000,17.000
Max_products,1.000,4.000
Lapse,0.000,7.000
Payment,0.000,1.000
Premium,40.140,2993.340
Cost_claims_year,0.000,260853.240



--- RESUM INICIAL ---
Registres: 105555
Variables: 30
Duplicats exactes: 0
Variables amb >10% nuls: ['Date_lapse']


#### <b>4.3.1.2 Conversió i validació de variables temporals</b>

In [6]:
# ============================================================
# Conversió i validació de variables temporals
# ============================================================
# Aquest script assumeix que el DataFrame `df` ja existeix.
# Realitza: conversió a datetime, validació de coherència,
# derivació de variables temporals bàsiques i exportació.
# ============================================================

# ---------------------------------------------
# 1. Identificar variables temporals
# ---------------------------------------------
# Llista de columnes que haurien de contenir dades de dates
date_cols = [
    'Date_start_contract',  # Data d’inici de la pòlissa
    'Date_last_renewal',    # Última renovació del contracte
    'Date_next_renewal',    # Següent renovació prevista
    'Date_birth',           # Data de naixement del titular
    'Date_driving_licence', # Data d’obtenció del carnet de conduir
    'Date_lapse'            # Data d’un possible sinistre o lapse temporal rellevant
]
# Mostrem per pantalla les columnes detectades (debug informatiu)
print("Variables temporals detectades:", date_cols)

# ---------------------------------------------
# 2. Conversió a datetime
# ---------------------------------------------
# Iterem per cada columna de data i convertim al tipus datetime64 de pandas
for col in date_cols:
    # format='%d/%m/%Y' indica que el format esperat és dia/mes/any (ex: 31/12/2020)
    # errors='coerce' força que qualsevol valor mal format es converteixi a NaT (valor nul temporal)
    df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')
# Comprovem els tipus resultants després de la conversió
print("\nTipus després de la conversió:")
print(df[date_cols].dtypes)  # Ha de mostrar datetime64[ns] per a totes
# Comptem quants valors s’han convertit a NaT degut a errors de conversió
print("\nValors NaT després de la conversió:")
print(df[date_cols].isna().sum())  # Resumeix quants nuls hi ha a cada data

# ---------------------------------------------
# 3. Validació temporal bàsica
# ---------------------------------------------
# Comptem anomalies temporals per validar que l'ordre cronològic sigui coherent
birth_anomaly = (df["Date_birth"] > df["Date_driving_licence"]).sum()
# → Casos on la data de naixement és posterior a la d'obtenció del carnet (impossible biològicament)
licence_anomaly = (df["Date_driving_licence"] > df["Date_last_renewal"]).sum()
# → Casos on el carnet s’obté després de la última renovació de la pòlissa (podria ser inconsistent)
contract_anomaly = (df["Date_start_contract"] > df["Date_last_renewal"]).sum()
# → Casos on la pòlissa comença *després* de la seva última renovació aparent (error clar)
next_negative = (df["Date_next_renewal"] < df["Date_last_renewal"]).sum()
# → Casos on la següent renovació és anterior a la última (inconsistència temporal)
lapse_before_contract = (df["Date_lapse"] < df["Date_start_contract"]).sum()
# → Casos on un lapse/sinistre ocorre *abans* que comenci la pòlissa (inconsistència)
lapse_before_last_renewal = (df["Date_lapse"] < df["Date_last_renewal"]).sum()
# → Casos on el lapse/sinistre és anterior a la darrera renovació (podria ser error o necessitar revisió)
# Mostrem totes les anomalies detectades
print("\n--- VALIDACIÓ TEMPORAL ---")
print("birth_anomaly:", birth_anomaly, "anomalies")
print("licence_anomaly:", licence_anomaly, "anomalies")
print("contract_anomaly:", contract_anomaly, "anomalies")
print("next_negative:", next_negative, "anomalies")
print("lapse_before_contract:", lapse_before_contract, "anomalies")
print("lapse_before_last_renewal:", lapse_before_last_renewal, "anomalies")

# ---------------------------------------------
# 4. Derivació de magnituds temporals bàsiques
# ---------------------------------------------
# Creem noves variables útils per analítica i models
df["Driver_age"] = (df["Date_last_renewal"] - df["Date_birth"]).dt.days / 365.25
# → Edat del conductor a la darrera renovació, en anys (aproximació amb 365.25 per considerar anys de traspàs)
df["Licence_age"] = (df["Date_last_renewal"] - df["Date_driving_licence"]).dt.days / 365.25
# → Anys d’antiguitat del carnet a la darrera renovació
df["Policy_age"] = (df["Date_last_renewal"] - df["Date_start_contract"]).dt.days / 365.25
# → Anys d’antiguitat de la pòlissa des de l’inici fins la darrera renovació
df["Days_to_next"] = (df["Date_next_renewal"] - df["Date_last_renewal"]).dt.days
# → Dies que falten fins la següent renovació des de la darrera
# Mostrem estadístiques resumides de les noves variables per veure distribucions bàsiques
print("\nDescripció estadística de les variables derivades:")
print(df[["Driver_age", "Licence_age", "Policy_age", "Days_to_next"]].describe())
# Mostrem una previsualització del dataset amb totes les dates i derivades per inspecció manual
print(df[[
    "Date_start_contract", "Date_last_renewal", "Date_next_renewal",
    "Date_birth", "Date_driving_licence", "Date_lapse",
    "Driver_age", "Licence_age", "Policy_age", "Days_to_next"
]].head())

# ---------------------------------------------
# 5. Exportació del dataset net
# ---------------------------------------------
# Guardem el DataFrame resultant a CSV
df.to_csv("clean_motor_insurance.csv", index=False, encoding="utf-8")
# → index=False evita afegir la columna d'índex a l'arxiu
# → encoding utf-8 assegura compatibilitat amb accents i caràcters especials
# Missatge de confirmació
print("\nFitxer exportat: clean_motor_insurance.csv")

Variables temporals detectades: ['Date_start_contract', 'Date_last_renewal', 'Date_next_renewal', 'Date_birth', 'Date_driving_licence', 'Date_lapse']

Tipus després de la conversió:
Date_start_contract     datetime64[ns]
Date_last_renewal       datetime64[ns]
Date_next_renewal       datetime64[ns]
Date_birth              datetime64[ns]
Date_driving_licence    datetime64[ns]
Date_lapse              datetime64[ns]
dtype: object

Valors NaT després de la conversió:
Date_start_contract         0
Date_last_renewal           0
Date_next_renewal           0
Date_birth                  0
Date_driving_licence        0
Date_lapse              70408
dtype: int64

--- VALIDACIÓ TEMPORAL ---
birth_anomaly: 0 anomalies
licence_anomaly: 31 anomalies
contract_anomaly: 68 anomalies
next_negative: 0 anomalies
lapse_before_contract: 0 anomalies
lapse_before_last_renewal: 404 anomalies

Descripció estadística de les variables derivades:
          Driver_age    Licence_age     Policy_age   Days_to_next
cou